In [1]:
from __future__ import print_function, division
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as im
from PIL import Image
import skimage
from tqdm import tqdm, tqdm_notebook
import re
import math

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils, datasets
import torchvision.models as models
from torch.autograd import Variable
from sklearn import linear_model
from torch import nn


%load_ext autoreload
%autoreload 2

In [2]:
data_dir = 'dataset/broden1_227/'
df = pd.read_csv(data_dir + 'processed_index.csv')
df.drop(df.columns[df.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True) # temp remove unnamed garbage col
df.insert(0, 'encoded', '')

In [3]:
def filter_by_category(df, category):
    pattern = "[%i]" % category
    return df[df['features'].str.contains(pattern, regex=False)]
    
def filter_by_not_category(df, category):
    pattern = "[%i]" % category
    return df[~df['features'].str.contains(pattern, regex=False)]

In [4]:
device = torch.device('cuda:0')

In [5]:
model = models.alexnet(pretrained=True)

model.to(device)

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Dropout(p=0.5)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
    (2): ReLU(inplace)
    (3): Dropout(p=0.5)
    (4): Linear(in_features=4096, out_feature

In [6]:
outputs = []

def hook(module, input, output):
    layer_output = output.data.cpu()
    outputs.append(layer_output)
        
model.features[6].register_forward_hook(hook) # register hook to access specific layer
model.eval()

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Dropout(p=0.5)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
    (2): ReLU(inplace)
    (3): Dropout(p=0.5)
    (4): Linear(in_features=4096, out_feature

In [7]:
class CategoryDataSet(Dataset):
    def __init__(self, df, img_dir, transform=None):
        self.dataframe = df
        self.root_dir = img_dir
        self.transform = transform
    
    def __len__(self):
        return len(self.dataframe)
    
    def __getitem__(self, idx):
        imgfile = os.path.join(self.root_dir,
                                self.dataframe.at[idx, 'image'])
        image = Image.open(imgfile)
        
        if self.transform:
            image = self.transform(image)
            
        sample = (image, idx)
        
        return sample

In [8]:
tf = transforms.Compose([transforms.Resize(224), transforms.ToTensor()])

dataset = CategoryDataSet(df, data_dir + 'images/', transform=tf)
dataloader = torch.utils.data.DataLoader(dataset, batch_size=30, shuffle=False, num_workers=1)

In [9]:
torch.cuda.empty_cache()

In [61]:
TRAIN_MAX_INDEX = 44403

In [10]:
outputs = []
predictions = []
# run dataset through model
for inputs, idx in tqdm(dataloader):
    inputs = inputs.to(device)
    model(inputs)

100%|██████████| 2111/2111 [04:10<00:00,  8.42it/s]


In [72]:
outputs[0].shape

torch.Size([30, 384, 13, 13])

In [11]:
stacked_outputs = torch.cat(outputs, dim=0)

In [12]:
m = nn.MaxPool2d(2)
max_pool_outputs = m(stacked_outputs)

In [13]:
max_pool_outputs.shape

torch.Size([63305, 384, 6, 6])

In [14]:
flattened_outputs = max_pool_outputs.reshape((max_pool_outputs.shape[0], np.prod(max_pool_outputs.shape[1:])))

In [15]:
# np.savetxt("flattened_pool_outputs.csv", flattened_outputs, delimiter=",")

In [16]:
flattened_outputs.shape

torch.Size([63305, 13824])

In [17]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import normalize
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC

In [60]:
pca = PCA(n_components=100)
normalized_outputs = normalize(flattened_outputs)
pca.fit(normalized_outputs[:44403]) # just do it on training data
np.sum(pca.explained_variance_ratio_)

0.5401557038159224

In [62]:
transformed_outputs = pca.transform(normalized_outputs)

In [63]:
df['encoded'] = transformed_outputs.tolist() # insert encodings to list

In [64]:
# df.to_csv(path_or_buf=data_dir + 'processed_index_with_encodings.csv')

In [65]:
SGD_MAX_ITERATIONS = 400
SGD_LOSS = "hinge"
SGD_PENALTY = "l2"

def train_clf_for_category(df, category):
    # Filter out category data from dataframe
    pos_df = filter_by_category(df, category)
    negative_df = filter_by_not_category(df,category).sample(len(pos_df), replace=True)
    
    if(len(pos_df) == 0):
        print("Empty set found for category %i" % category)
        return(None, 0)
    
    # generate training and validation data for SGDClassifier
    train = pos_df[pos_df.split == 'train'].encoded.to_list() + negative_df.sample(len(pos_df[pos_df.split == 'train'])).encoded.to_list()
    val = pos_df[pos_df.split == 'val'].encoded.to_list() + negative_df.sample(len(pos_df[pos_df.split == 'val'])).encoded.to_list()
        
    # scale data
#     scaler = StandardScaler()
#     scaler.fit(train)  # Don't cheat - fit only on training data
#     train = scaler.transform(train)
#     val = scaler.transform(val)  # apply same transformation to test data
    
    train_labels = np.append(np.ones(len(train) // 2), np.zeros(len(train) // 2))
    val_labels = np.append(np.ones(len(val) // 2), np.zeros(len(val) // 2))
    clf = linear_model.SGDClassifier(max_iter=min(np.ceil(10**6 / len(train_labels)), SGD_MAX_ITERATIONS), loss=SGD_LOSS, early_stopping=True, penalty=SGD_PENALTY, average=True, eta0=1.5)
#     clf = LinearSVC(random_state=0, tol=1e-5)
    clf.fit(train, train_labels)
    return (clf, clf.score(val, val_labels))
    return None, 0


In [67]:
clf, score = train_clf_for_category(df, 1)
print(score)

0.8962228466443316


In [68]:
# chosen based on frequency in the dataset
MAX_CLASSES = 150
SCORE_THRESH = 0.75
sgd_classifiers = {}

In [69]:
for i in tqdm(range(1, MAX_CLASSES)):
    clf, score = train_clf_for_category(df, i)
    if(score > SCORE_THRESH):
        sgd_classifiers[i] = (clf, score)
    else:
        print("ignoring clf for category %i with score %i", (i, score))


  5%|▍         | 7/149 [00:18<06:28,  2.73s/it]

ignoring clf for category %i with score %i (7, 0.7215756763343895)



  8%|▊         | 12/149 [00:30<05:22,  2.36s/it]

ignoring clf for category %i with score %i (12, 0.6311242344706912)



  9%|▉         | 14/149 [00:34<04:24,  1.96s/it]

...


 99%|█████████▊| 147/149 [03:33<00:01,  1.10it/s]

ignoring clf for category %i with score %i (147, 0.6813186813186813)



 99%|█████████▉| 148/149 [03:33<00:00,  1.13it/s]

ignoring clf for category %i with score %i (148, 0.7194444444444444)



100%|██████████| 149/149 [03:34<00:00,  1.20it/s]

In [70]:
import pickle

# Store data (serialize)
with open('classifiers.pickle', 'wb') as handle:
    pickle.dump(sgd_classifiers, handle, protocol=pickle.HIGHEST_PROTOCOL)



In [71]:
len (sgd_classifiers)

88